In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy as sp
from scipy import linalg as la
import scipy.cluster.hierarchy as hier

from sklearn.manifold import TSNE
from collections import Counter
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


In [2]:
def my_alignment(input):
    if on == 0:
        return input[0:input.shape[0] - 1]

    input_images = input[0:input.shape[0] - 1]
    w_image = input[input.shape[0] - 1]
    # org_shape = w_image.shape

    # imgs1 = np.array(input_images * 255, dtype=np.float32)
    # img2 = np.array(w_image * 255, dtype=np.float32)

    imgs1 = np.array(input_images, dtype=np.float32)
    img2 = np.array(w_image, dtype=np.float32)

    # stretch = 1
    # imgs1 = [cv2.resize(x, (stretch * x.shape[1], stretch * x.shape[0]), interpolation=cv2.INTER_CUBIC) for x in imgs1]
    # imgs1 = [cv2.threshold(x, 128, 255, cv2.THRESH_BINARY)[1] for x in imgs1]
    # imgs1 = [cv2.Laplacian(x, cv2.CV_8U) for x in imgs1]
    # img2 = cv2.resize(img2, (stretch * w_image.shape[1], stretch * w_image.shape[0]), interpolation=cv2.INTER_CUBIC)
    # img2 = cv2.threshold(img2, 128, 255, cv2.THRESH_BINARY)[1]
    # img2 = cv2.Laplacian(img2, cv2.CV_8U)

    # Find size of image1
    sz = imgs1[0].shape
    # Specify the number of iterations.
    # number_of_iterations = 5000;
    number_of_iterations = 50;

    # Specify the threshold of the increment
    # in the correlation coefficient between two iterations
    # termination_eps = 1e-10;
    termination_eps = 0.001;

    # Define the motion model
    # warp_mode = cv2.MOTION_AFFINE
    warp_mode = cv2.MOTION_EUCLIDEAN
    # warp_mode = cv2.MOTION_HOMOGRAPHY

    transformedImages = []
    for i in range(0, len(input_images)):
        # Define 2x3 or 3x3 matrices and initialize the matrix to identity
        if warp_mode == cv2.MOTION_HOMOGRAPHY:
            warp_matrix = np.eye(3, 3, dtype=np.float32)
        else:
            warp_matrix = np.eye(2, 3, dtype=np.float32)

        # Define termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations, termination_eps)

        try:
            # Run the ECC algorithm. The results are stored in warp_matrix.
            (cc, warp_matrix) = cv2.findTransformECC(img2, imgs1[i], warp_matrix, warp_mode, criteria)

            if warp_mode == cv2.MOTION_HOMOGRAPHY:
                # Use warpPerspective for Homography
                im1_aligned = cv2.warpPerspective(imgs1[i], warp_matrix, (sz[1], sz[0]),
                                                  flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
            else:
                # Use warpAffine for Translation, Euclidean and Affine
                im1_aligned = cv2.warpAffine(imgs1[i], warp_matrix, (sz[1], sz[0]),
                                             flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

                # im1_aligned = cv2.resize(im1_aligned, (org_shape[1],org_shape[0]), interpolation=cv2.INTER_CUBIC)
                # print("Done")
        except cv2.error:
            im1_aligned = imgs1[i]
            # im1_aligned = cv2.resize(im1_aligned, (org_shape[1], org_shape[0]), interpolation=cv2.INTER_CUBIC)
            # print("error while trying to find alignment")

        transformedImages.append(im1_aligned)
        # Show final results
        # cv2.imshow("Image 1", imgs1[i])
        # cv2.imshow("Image 2", img2)
        # cv2.imshow("Aligned Image 2", im2_aligned)
        # cv2.waitKey(0)
    transformedImages = np.reshape(transformedImages, [-1, 28, 28])
    # print(transformedImages.shape)
    # return transformedImages.astype(np.float32)
    return transformedImages
    # return input[0:input.shape[0]-1]


def my_get_wieghts(input_data, name=None):
    kernel = tf.Variable(tf.zeros([10, 784]), name=name)
    input_shaped = tf.reshape(input_data, [-1, 28, 28])

    res = []
    ws = []
    for k in range(kernel.shape[0].value):
        W = tf.reshape(kernel[k], [28, 28])

        # Rx = tf.py_func(my_alignment, [input_shaped, W], tf.float32)

        Rx = tf.py_func(my_alignment, [tf.concat([input_shaped, tf.reshape(W, [-1, 28, 28])], 0)], tf.float32)
        Rx = tf.stop_gradient(Rx)

        # stackedInputAndR = tf.stack([input_shaped, R])
        # transposedInputAndR = tf.transpose(stackedInputAndR, perm=[1,0,2,3])
        # Rx = tf.map_fn(lambda x: tf.matmul(x[1], x[0]), transposedInputAndR)
        Rx = tf.reshape(Rx, [-1, 784])
        W = tf.reshape(W, [784, 1])
        Y = tf.matmul(Rx, W)
        res.append(Y)

    return tf.reshape(tf.transpose(res), [-1, 10]), kernel


In [3]:
for model_id in ['0', '100', '200', '300', '400', '500', '600', '700', '800', '900']:
    model_checkpoint_file = 'Models\model_{}.ckpt'.format(model_id)
    meta_graph_file = model_checkpoint_file + '.meta'

    x = tf.placeholder(tf.float32, [None, 784])
    #Wx, kernel = my_get_wieghts(x, name="proc_wieghts")
    W = tf.Variable(tf.zeros([784, 10]))
    Wx = tf.matmul(x, W)
    b = tf.Variable(tf.zeros([10]))
    y = Wx + b

    sess = tf.Session()
    saver = tf.train.import_meta_graph(meta_graph_file)
    saver.restore(sess, model_checkpoint_file)

    variables = tf.global_variables()
    plt.figure(figsize=(16,16))
    for i in range(10):
        plt.subplot(3,4,i+1)
        plt.imshow(np.reshape(variables[0].eval(sess)[i,:], (28,28)), cmap='gray')
    plt.savefig('Models\kernel_{}.png'.format(model_id))


INFO:tensorflow:Restoring parameters from Models\model_0.ckpt


FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: _retval_Variable_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable)]]

In [15]:
model_checkpoint_file = 'Models\model_{}.ckpt'.format('200')

saver.restore(sess, model_checkpoint_file)
tf.global_variables()[0].eval(sess)

INFO:tensorflow:Restoring parameters from Models\model_200.ckpt


FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: _retval_Variable_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable)]]